# Testing IBMi + Jupyter Notebooks

Using Jupyter cell magic to execute DB2 for i queries

### Configure IBMi Hosts

In [4]:
import pyodbc, getpass
import pandas as pd
from ipywidgets import widgets, Text, HBox
from IPython.display import display
from IPython.core.magic import register_cell_magic

# disable pandas df display limit, trust user to limit query results
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

#HOST_OPTIONS = ['PUB400.COM'] # TODO move to widget?

w_host_list = widgets.Text(placeholder='Enter hosts (comma delimited)', description='Hosts: ')
display(w_host_list)

Text(value='', description='Hosts: ', placeholder='Enter hosts (comma delimited)')

### Enter Credentials

In [14]:
host_list = [h for h in w_host_list.value.split(',') if len(h.strip()) > 0]

if len(host_list[0]) > 0:
    w_host = widgets.Dropdown(options=host_list, value=host_list[0], description='Host: ')
else:
    w_host = widgets.Text(placeholder='Enter host', description='Host: ')
w_user = widgets.Text(placeholder='Enter user', description='User: ')
w_pwd = widgets.Password(placeholder='Enter password', description='Password: ')

display(HBox((w_host, w_user, w_pwd)))

### Notebook setup
(You shouldn't have to edit this)

In [34]:
host,user,pwd = w_host.value.strip(), w_user.value.strip(), w_pwd.value
conn = pyodbc.connect(driver='{IBM i Access ODBC Driver}', system=host, uid=user, pwd=pwd)
print("Successfully connected to host '{}' as user '{}'".format(host, user))

# Add IBMi DB2 SQL magic: invoke using %%sql
@register_cell_magic
def db2i(line, cell):
    line = line.replace("'", "\\'").replace('"', '\\"')
    cell = cell.format(**globals())
    if line.strip() != '-':
        res = pd.read_sql(cell, conn)
        if line.strip() != '': 
            get_ipython().user_ns[line.strip()] = res
        return res
    else:
        conn.execute(cell)
del db2i

Successfully connected to host 'PUB400.COM' as user 'OTTEB'


In [36]:
%%db2i 

select * 
from QSYS2.SYSTABLES
where TABLE_OWNER='OTTEB'
limit 10

,TABLE_NAME,TABLE_OWNER,TABLE_TYPE,COLUMN_COUNT,ROW_LENGTH,TABLE_TEXT,LONG_COMMENT,TABLE_SCHEMA,LAST_ALTERED_TIMESTAMP,SYSTEM_TABLE_NAME,SYSTEM_TABLE_SCHEMA,FILE_TYPE,BASE_TABLE_CATALOG,BASE_TABLE_SCHEMA,BASE_TABLE_NAME,BASE_TABLE_MEMBER,SYSTEM_TABLE,SELECT_OMIT,IS_INSERTABLE_INTO,IASP_NUMBER,ENABLED,MAINTENANCE,REFRESH,REFRESH_TIME,MQT_DEFINITION,ISOLATION,PARTITION_TABLE,TABLE_DEFINER,MQT_RESTORE_DEFERRED,ROUNDING_MODE,CONTROL,TEMPORAL_TYPE
0,QCMDSRC,OTTEB,P,3,92,CMD Source,None,OTTEB1,2020-01-29 15:15:17.166,QCMDSRC,OTTEB1,S,None,None,None,None,N,N,YES,0,None,None,None,None,None,None,NO,OTTEB,None,None,,N
1,QCLLESRC,OTTEB,P,3,92,CLLE Source,None,OTTEB1,2020-01-29 14:54:40.727,QCLLESRC,OTTEB1,S,None,None,None,None,N,N,YES,0,None,None,None,None,None,None,NO,OTTEB,None,None,,N
2,QDDSSRC,OTTEB,P,3,132,,None,OTTEB1,2020-01-26 02:24:59.341,QDDSSRC,OTTEB1,S,None,None,None,None,N,N,YES,0,None,None,None,None,None,None,NO,OTTEB,None,None,,N
3,QRPGLESRC,OTTEB,P,3,92,,None,OTTEB1,2020-01-26 02:24:59.341,QRPGLESRC,OTTEB1,S,None,None,None,None,N,N,YES,0,None,None,None,None,None,None,NO,OTTEB,None,None,,N
4,TODOLIST,OTTEB,P,7,192,,None,OTTEB1,2020-02-02 19:37:16.942,TODOLIST,OTTEB1,D,None,None,None,None,N,N,YES,0,None,None,None,None,None,None,NO,OTTEB,None,None,,N


In [33]:
%%db2i

select 
  id,
  username,
  url,
  (minutes / 60) as hours_watched
from json_table(
  Systools.HttpPostClob(
    'https://graphql.anilist.co',
    cast('<httpHeader><header name="Content-Type" value="application/json"/></httpHeader>' as clob),
    cast('{"query": "{User(search:\"barrettotte\"){id name siteUrl stats{watchedTime}}}"}' as clob)
  ),
  '$.data.User'
  columns(
    id       char(10) path '$.id',
    username char(20) path '$.name',
    url      char(32) path '$.siteUrl',
    minutes  char(10) path '$.stats.watchedTime'
  )
);



KeyError: '"query"'

## Gotchas
* In cells with ```%%db2i```, you can't mix python and SQL. Whatever the cell detects first is what it runs